In [ ]:
import os, sys
sys.path.append('/root/catkin_ws/src/primitives/')
import pickle
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import trimesh
import networkx

from open3d import JVisualizer

import copy
import time
import argparse
import numpy as np
from multiprocessing import Process, Pipe, Queue
import pickle
import rospy
import copy
import signal
import open3d
from IPython import embed

from yacs.config import CfgNode as CN
from closed_loop_experiments import get_cfg_defaults

from airobot import Robot
from airobot.utils import pb_util
from airobot.sensor.camera.rgbdcam_pybullet import RGBDCameraPybullet
from airobot.utils import common
import pybullet as p

from helper import util
from macro_actions import ClosedLoopMacroActions, YumiGelslimPybulet
# from closed_loop_eval import SingleArmPrimitives, DualArmPrimitives

In [ ]:
sys.path.append('/root/training/')

import os
import argparse
import time
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from data_loader import DataLoader
from model import VAE
from util import to_var, save_state, load_net_state, load_seed, load_opt_state

In [ ]:
# with open ('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/2.pkl', 'rb') as f:
#     grasp_data = pickle.load(f)
    
# print(grasp_data.keys())

In [ ]:
# with open ('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0_fixed/1003.pkl', 'rb') as f:
#     pull_data = pickle.load(f)
got_file = False
while not got_file:
    val = np.random.randint(low=0, high=2000)
    test_path = '/root/catkin_ws/src/primitives/data/pull/face_ind_large_0_fixed/%d.pkl' % val
#     test_path = '/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/%d.pkl' % val
    if os.path.exists(test_path):
        with open (test_path, 'rb') as f:
            pull_data = pickle.load(f)
#             grasp_data = pickle.load(f)
            got_file = True

print(test_path)
print(pull_data.keys())
# print(grasp_data.keys())

# Training data: 
## Inputs: 
Initial observation, different representations:
- ```'start'```: pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```
- ```'keypoints_start'```: 3D location of box corners at start pose
- ```'obs':'pcd_pts'```: Point cloud of box from 3 different viewpoints, with same global coordinate sys. Can be fused with np.concatenate (see below)

Goal, different representations:
- ```'goal'```: pose
- ```'keypoints_goal'```: 3D location of box corners at goal pose

In [ ]:
# start observation
start = pull_data['start']
keypoints_start = pull_data['keypoints_start']
pcd_pts = pull_data['obs']['pcd_pts']
pcd_pts_start = np.concatenate(pcd_pts, axis=0)

# goal
goal = pull_data['goal']
keypoints_goal = pull_data['keypoints_goal']

# start observation
# start = grasp_data['start']
# keypoints_start = grasp_data['keypoints_start']
# pcd_pts = grasp_data['obs']['pcd_pts']
# pcd_pts_start = np.concatenate(pcd_pts, axis=0)

# # goal
# goal = grasp_data['goal']
# keypoints_goal = grasp_data['keypoints_goal']

# Outputs:
## Pulling/Pushing (single arm)
Robot palm pose in the object frame, for active arm -- active arm currently based on which side of the table the object starts on. TODO perhaps includes predicting which arm is active, once we move to more diverse data. For now, everything for pulling happens with the right arm
- ```'contact_obj_frame'```: pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose

## Grasping/Pivoting (dual arm)

Right and left robot palm pose in the object frame
- ```'contact_obj_frame':'right'```: right palm pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose
- ```'contact_obj_frame':'left'```: left palm pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose

In [ ]:
# right palm contact for pulling
contact_r = pull_data['contact_obj_frame']

# both palms contact for grasping
# contact_r = grasp_data['contact_obj_frame']['right']
# contact_l = grasp_data['contact_obj_frame']['left']

# Training

In [ ]:
## helpers

# load minibatch
# def load_minibatch(M, datatype='pose')


## setup

In [ ]:
# load model architecture (encoder and decoder)
# setup optimizer

# setup loss function

## train loop

# for epoch in num_epochs:
# for minibatch in minibatch_size / data_size:
# forward pass, compute loss, backprop, optimizer.step
# look at loss

In [ ]:
def pose_to_list(palm_pose_tensor):
    pos = palm_pose_tensor[:3].data.cpu().numpy()
    ori = palm_pose_tensor[3:].data.cpu().numpy()
    
    ori = ori/np.linalg.norm(ori)
    
    return pos.tolist() + ori.tolist()

# Testing

In [ ]:
## setup

# distribution to sample from for start/goal (same as in training)

# load trained model

# vae = VAE(14, 7, 2, 0.0003)
# load_net_state(vae, '/root/training/saved_models/pose_init_small_batch_1/pose_init_small_batch_1_epoch_63.pt')

vae = VAE(31, 7, 3, 0.0003)
load_net_state(vae, '/root/training/saved_models/pull_keypoints_init_small_batch_0/pull_keypoints_init_small_batch_0_epoch_99.pt')

# vae = VAE(14, 14, 3, 0.0003)
# load_net_state(vae, '/root/training/saved_models/grasp_pose_init_small_batch_0/grasp_pose_init_small_batch_0_epoch_298.pt')

# vae = VAE(31, 14, 3, 0.0003)
# load_net_state(vae, '/root/training/saved_models/grasp_keypoints_init_small_batch_0/grasp_keypoints_init_small_batch_0_epoch_26.pt')


## eval

In [ ]:
# sample from the latent space and use the decoder/generator to produce contacts, visualize below
# x = torch.from_numpy(np.asarray(start+goal, dtype=np.float32))

x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))

z, pose_sample, z_mu, z_logvar = vae.forward(x)

In [ ]:
print("mu: " + str(z_mu) + ", std: " + str(torch.exp(0.5*z_logvar)))

In [ ]:
# pose = vae.decode(z)
pose = vae.decode(z_mu)
print(pose)

In [ ]:
pose_r = pose[:7]
pose_l = pose[7:]
# pose = vae.decode(torch.normal(z_mu, torch.exp(0.5*z_logvar)))

In [ ]:
pos = pose[:3].data.cpu().numpy()
ori = pose[3:].data.cpu().numpy()

# pos = pose_r[:3].data.cpu().numpy()
# ori = pose_r[3:].data.cpu().numpy()

In [ ]:
ori = ori/np.linalg.norm(ori)

In [ ]:
print(pos)
print(ori)

In [ ]:
pull_data_eval = copy.deepcopy(pull_data)
pull_data_eval['contact_obj_frame'] = pos.tolist() + ori.tolist()

# grasp_data_eval = copy.deepcopy(grasp_data)
# grasp_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# grasp_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

In [ ]:
print("data: " + str(pull_data['contact_obj_frame']))
print("--------")
print("eval: " + str(pos.tolist() + ori.tolist()))

# print("right: " + str(grasp_data_eval['contact_obj_frame']['right']))
# print("----------------------------")
# print("left: " + str(grasp_data_eval['contact_obj_frame']['left']))

# Visualization

In [ ]:
with open('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0/metadata.pkl', 'rb') as mf:
    metadata = pickle.load(mf)
# with open('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/metadata.pkl', 'rb') as mf:
#     metadata = pickle.load(mf)


print('Metadata keys: ')
dynamics_info = metadata['dynamics']
mesh_file = metadata['mesh_file']
palm_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/mpalm/mpalms_all_coarse.stl'
table_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/table/table_top.stl'
cfg = metadata['cfg']

# Visualize contact on object

In [ ]:
def vis_palms(data, name='pull'):
    obj_mesh = trimesh.load_mesh(mesh_file)
    r_palm_mesh = trimesh.load_mesh(palm_mesh_file)
    l_palm_mesh = trimesh.load_mesh(palm_mesh_file)
    table_mesh = trimesh.load_mesh(table_mesh_file)
    
    obj_pos_world = data['start'][:3]
    obj_ori_world = data['start'][3:]
    obj_ori_mat = common.quat2rot(obj_ori_world)
    h_trans = np.zeros((4, 4))
    h_trans[:3, :3] = obj_ori_mat
    h_trans[:-1, -1] = obj_pos_world
    h_trans[-1, -1] = 1

    obj_mesh.apply_transform(h_trans)
    if name == 'pull':
        tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame'])
        tip_contact_r = util.convert_reference_frame(
            pose_source=tip_contact_r_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))

        wrist_contact_r = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_r)

        wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
        
        palm_pos_world_r = wrist_contact_r_list[:3]
        palm_ori_world_r = wrist_contact_r_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_r)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_r
        h_trans[-1, -1] = 1

        r_palm_mesh.apply_transform(h_trans)      
        
        scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh])        
    else:
        tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame']['right'])
        tip_contact_l_obj = util.list2pose_stamped(data['contact_obj_frame']['left'])

        tip_contact_r = util.convert_reference_frame(
            pose_source=tip_contact_r_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))
            
        tip_contact_l = util.convert_reference_frame(
            pose_source=tip_contact_l_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))            
            
        wrist_contact_r = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_r)

        wrist_contact_l = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_l)

        wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
        wrist_contact_l_list = util.pose_stamped2list(wrist_contact_l)
        
        palm_pos_world_r = wrist_contact_r_list[:3]
        palm_ori_world_r = wrist_contact_r_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_r)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_r
        h_trans[-1, -1] = 1

        r_palm_mesh.apply_transform(h_trans)
        
        palm_pos_world_l = wrist_contact_l_list[:3]
        palm_ori_world_l = wrist_contact_l_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_l)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_l
        h_trans[-1, -1] = 1

        l_palm_mesh.apply_transform(h_trans)        
        
        scene = trimesh.Scene([obj_mesh, r_palm_mesh, l_palm_mesh, table_mesh])
    return scene

In [ ]:
def correct_pos(data):
    contact_obj_frame_pred = util.list2pose_stamped(pull_data_eval['contact_obj_frame'])
    contact_world_frame = util.convert_reference_frame(contact_obj_frame_pred, util.unit_pose(), util.list2pose_stamped(start))
    contact_pos = open3d.utility.DoubleVector(np.array(util.pose_stamped2list(contact_world_frame)[:3]))

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_pts']))
    pcd.colors = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_colors']) / 255.0)

    kdtree = open3d.geometry.KDTreeFlann(pcd)
    nearest_pt_ind = kdtree.search_knn_vector_3d(contact_pos, 1)[1][0]
    nearest_pt_world = np.asarray(pcd.points)[nearest_pt_ind]

    contact_world_frame_corrected = copy.deepcopy(contact_world_frame)
    contact_world_frame_corrected.pose.position.x = nearest_pt_world[0]
    contact_world_frame_corrected.pose.position.y = nearest_pt_world[1]
    contact_world_frame_corrected.pose.position.z = nearest_pt_world[2]

    contact_obj_frame_corrected = util.pose_stamped2list(util.convert_reference_frame(contact_world_frame_corrected, util.list2pose_stamped(start), util.unit_pose()))
    new_data = copy.deepcopy(data)
    new_data['contact_obj_frame'] = contact_obj_frame_corrected
    return new_data

In [ ]:
scene = vis_palms(pull_data_eval, name='pull')
# scene = vis_palms(grasp_data_eval, name='grasp')

scene.show(viewer='gl')

In [ ]:
scene = vis_palms(correct_pos(pull_data_eval), name='pull')
# scene = vis_palms(grasp_data, name='grasp')

scene.show(viewer='gl')

In [ ]:
x = torch.from_numpy(np.asarray(start+goal, dtype=np.float32))
output = vae.forward(x)

pos = output[1][:3].data.cpu().numpy()
ori = output[1][3:].data.cpu().numpy()

ori = ori/np.linalg.norm(ori)

pull_data_eval = copy.deepcopy(pull_data)
pull_data_eval['contact_obj_frame'] = pos.tolist() + ori.tolist()

scene = vis_palms(pull_data_eval, name='pull')
# scene = vis_palms(grasp_data, name='grasp')

scene.show(viewer='gl')

In [ ]:
print(pull_data['contact_obj_frame'])
print(pull_data_eval['contact_obj_frame'])

In [ ]:
1 - np.dot(pull_data['contact_obj_frame'], pull_data_eval['contact_obj_frame'])**2

In [ ]:
outputs = []
z = torch.randn_like(torch.zeros(500, 3))

output = vae.decode(z)

In [ ]:
output_mean = []
output_std = []
for i in range(output.shape[1]):
    avg_dim_i = torch.mean(output[:, i], axis=0)
    std_dim_i = torch.std(output[:, i], axis=0)
    print(avg_dim_i)
    output_mean.append(avg_dim_i.data.cpu())
    output_std.append(std_dim_i.data.cpu())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math

%matplotlib notebook

In [ ]:
dim = 2
sigma = output_std[dim]
mu = output_mean[dim]

x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.show()

In [ ]:
# pos = np.asarray(output_mean[:3])
# ori = np.asarray(output_mean[3:])

# ori = ori/np.linalg.norm(ori)

# pull_data_eval = copy.deepcopy(pull_data)
# pull_data_eval['contact_obj_frame'] = pos.tolist() + ori.tolist()

# scene = vis_palms(pull_data_eval, name='pull')

pose_r = output_mean[:7]
pose_l = output_mean[7:]

grasp_data_eval = copy.deepcopy(grasp_data)
grasp_data_eval['contact_obj_frame']['right'] = pose_r
grasp_data_eval['contact_obj_frame']['left'] = pose_l
scene = vis_palms(grasp_data_eval, name='grasp')

scene.show(viewer='gl')

In [ ]:
def correct_pos(data):
    contact_obj_frame_pred = util.list2pose_stamped(pull_data_eval['contact_obj_frame'])
    contact_world_frame = util.convert_reference_frame(contact_obj_frame_pred, util.unit_pose(), util.list2pose_stamped(start))
    contact_pos = open3d.utility.DoubleVector(np.array(util.pose_stamped2list(contact_world_frame)[:3]))

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_pts']))
    pcd.colors = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_colors']) / 255.0)

    kdtree = open3d.geometry.KDTreeFlann(pcd)
    nearest_pt_ind = kdtree.search_knn_vector_3d(contact_pos, 1)[1][0]
    nearest_pt_world = np.asarray(pcd.points)[nearest_pt_ind]

    contact_world_frame_corrected = copy.deepcopy(contact_world_frame)
    contact_world_frame_corrected.pose.position.x = nearest_pt_world[0]
    contact_world_frame_corrected.pose.position.y = nearest_pt_world[1]
    contact_world_frame_corrected.pose.position.z = nearest_pt_world[2]

    contact_obj_frame_corrected = util.pose_stamped2list(util.convert_reference_frame(contact_world_frame_corrected, util.list2pose_stamped(start), util.unit_pose()))
    return contact_obj_frame_correctedrected

In [ ]:
contact_obj_frame_pred = util.list2pose_stamped(pull_data_eval['contact_obj_frame'])
contact_world_frame = util.convert_reference_frame(contact_obj_frame_pred, util.unit_pose(), util.list2pose_stamped(start))
contact_pos = open3d.utility.DoubleVector(np.array(util.pose_stamped2list(contact_world_frame)[:3]))

pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_pts']))
pcd.colors = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_colors']) / 255.0)

kdtree = open3d.geometry.KDTreeFlann(pcd)
nearest_pt_ind = kdtree.search_knn_vector_3d(contact_pos, 1)[1][0]
nearest_pt_world = np.asarray(pcd.points)[nearest_pt_ind]

contact_world_frame_corrected = copy.deepcopy(contact_world_frame)
contact_world_frame_corrected.pose.position.x = nearest_pt_world[0]
contact_world_frame_corrected.pose.position.y = nearest_pt_world[1]
contact_world_frame_corrected.pose.position.z = nearest_pt_world[2]

contact_obj_frame_corrected = util.pose_stamped2list(util.convert_reference_frame(contact_world_frame_corrected, util.list2pose_stamped(start), util.unit_pose()))

In [ ]:
print(pull_data_eval['contact_obj_frame'])
print(contact_obj_frame_corrected)

In [ ]:
pull_data_eval['contact_obj_frame'] = contact_obj_frame_corrected
scene = vis_palms(pull_data_eval, name='pull')
# scene = vis_palms(grasp_data, name='grasp')

scene.show(viewer='gl')

In [ ]:
import rospy
from airobot import Robot
from closed_loop_eval import SingleArmPrimitives

In [ ]:
rospy.init_node("test")

In [ ]:
yumi = Robot('yumi_palms', arm_cfg={'render': True, 'self_collision': False, 'rt_simulation': True})

In [ ]:
yumi.arm.go_home()

In [ ]:
yumi.arm.set_jpos(cfg.RIGHT_INIT + cfg.LEFT_INIT)

In [ ]:
gel_id = 12

p.changeDynamics(
    yumi.arm.robot_id,
    gel_id,
    restitution=dynamics_info['restitution'],
    contactStiffness=dynamics_info['contactStiffness'],
    contactDamping=dynamics_info['contactDamping'],
    rollingFriction=dynamics_info['rollingFriction']
)

In [ ]:
yumi_gs = YumiGelslimPybulet(
    yumi,
    cfg)

In [ ]:
box_id = pb_util.load_urdf(
    '/root/catkin_ws/src/config/descriptions/urdf/realsense_box.urdf',
    cfg.OBJECT_POSE_3[0:3],
    cfg.OBJECT_POSE_3[3:]
)

In [ ]:
exp_single = SingleArmPrimitives(cfg, box_id, mesh_file)

config_pkg_path = '/root/catkin_ws/src/config/'

action_planner = ClosedLoopMacroActions(
    cfg,
    yumi_gs,
    box_id,
    pb_util.PB_CLIENT,
    config_pkg_path,
    object_mesh_file=mesh_file,
    replan=False
)

# planner_args = data['planner_args']
object_start_pose_list = pull_data['start']
object_goal_pose_list = pull_data['goal']

pb_util.reset_body(
    body_id=box_id, 
    base_pos=object_start_pose_list[:3],
    base_quat=object_start_pose_list[3:])

exp_single.transform_mesh_world()
keypoints_start = np.array(exp_single.mesh_world.vertices.tolist())

In [ ]:
new_args = {}
new_args['object_pose1_world'] = util.list2pose_stamped(pull_data['start'])
new_args['object_pose2_world'] = util.list2pose_stamped(pull_data['goal'])
new_args['object_pose2_world'].pose.position.z /= 1.05
new_args['primitive_name'] = 'pull'
# new_args['palm_pose_r_object'] = util.list2pose_stamped(data['contact_obj_frame'])
# new_args['palm_pose_l_object'] = util.list2pose_stamped(cfg.PALM_LEFT)
# new_args['palm_pose_r_object'] = util.list2pose_stamped(data['contact_obj_frame']['right'])
# new_args['palm_pose_l_object'] = util.list2pose_stamped(data['contact_obj_frame']['left'])
new_args['object'] = None
new_args['init'] = True
new_args['N'] = 60
new_args['table_face'] = 0

In [ ]:
# new_args['palm_pose_r_object'] = util.list2pose_stamped(data['contact_obj_frame']['right'])
# new_args['palm_pose_l_object'] = util.list2pose_stamped(data['contact_obj_frame']['left'])
x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))

# palm_pose = vae.forward(x)[1].data.cpu().numpy()
# x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))

z, pose_sample, z_mu, z_logvar = vae.forward(x)
palm_pose = vae.decode(z_mu)

pos = palm_pose[:3].data.cpu().numpy()
ori = palm_pose[3:].data.cpu().numpy()
ori = ori/np.linalg.norm(ori)

In [ ]:
new_args['palm_pose_r_object'] = util.list2pose_stamped(pos.tolist() + ori.tolist())
new_args['palm_pose_l_object'] = util.list2pose_stamped(cfg.PALM_LEFT)

In [ ]:
print(pos)
print(ori)

In [ ]:
planner_args = new_args
primitive_name = planner_args['primitive_name']

In [ ]:
result = action_planner.execute(primitive_name=primitive_name, execute_args=planner_args)

In [ ]:
pb_util.remove_body(box_id)

In [ ]:
# yumi_gs.yumi_pb.arm.go_home()

In [ ]:
# jnts = yumi_gs.compute_ik(pos.tolist(), ori.tolist(), yumi_gs.get_jpos()[:7], arm='right')

In [ ]:
# yumi_gs.update_joints(list(jnts) + yumi_gs.get_jpos()[7:])

In [ ]:
yumi_gs.update_joints(cfg.RIGHT_INIT + cfg.LEFT_INIT)